In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("./model/bert_base_chinese")
tokenizer = AutoTokenizer.from_pretrained("./model/bert_base_chinese")

sentences = [
    'null',
    'Input-Asr-None',
    '(side)你看他一来就行我们那么的不会用搞来搞去的不知道是去哪个码头(unknown)',
    '这也是一句话',
    '这是最后一句话',
]
sentences = [' '.join(sentence) for sentence in sentences]

inputs = tokenizer(sentences, padding=True, return_tensors='pt')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
print(input_ids.shape, attention_mask.shape)
outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True).hidden_states[-1]
print(outputs.shape)

# output each token's corresponding word piece
for input in inputs['input_ids']:
    for token in input:
        print(tokenizer.decode([token.item()]), end=' ')
    print()

Some weights of the model checkpoint at ./model/bert_base_chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([5, 46]) torch.Size([5, 46])
torch.Size([5, 46, 768])
[CLS] n u l l [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
[CLS] i n p u t - a s r - n o n e [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
[CLS] ( s i d e ) 你 看 他 一 来 就 行 我 们 那 么 的 不 会 用 搞 来 搞 去 的 不 知 道 是 去 哪 个 码 头 ( u n k n o w n ) [SEP] 
[CLS] 这 也 是 一 句 话 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
[CLS] 这 是 最 后 一 句 话 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [